# Final Project
CS696 Big Data

Professor Whitney

Team:

Kristi Werry - 823386935

William Ritchie - 815829203

## Description

## Imports

In [49]:
import numpy as np
import pandas as pd
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.types import BooleanType
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.functions import lit
import sys
from pyspark.sql import SparkSession

## Functions

In [50]:
#takes in the dataset and a country
#converts the columns to the proper types and add the country column
def castTypes_addCountry(data, country, columns):
    data_processed =  data.withColumn("views", data["views"]
                      .cast(IntegerType())).withColumn("likes", data["likes"]
                      .cast(IntegerType())).withColumn("dislikes", data["dislikes"]
                      .cast(IntegerType())).withColumn("comment_count", data["comment_count"]
                      .cast(IntegerType())).withColumn("comments_disabled", data["comments_disabled"]
                      .cast(BooleanType())).withColumn("ratings_disabled", data["ratings_disabled"]
                      .cast(BooleanType())).withColumn("video_error_or_removed", data["video_error_or_removed"]
                      .cast(BooleanType())).withColumn("trending_date", to_timestamp(data["trending_date"], 
                      "yy.dd.MM")).withColumn("publish_time", to_timestamp(data["publish_time"], 
                      "yyyy-MM-dd")).withColumn("country", lit(country)).select(columns)
    return data_processed

## Importing Datasets

In [51]:
sqlContext = SparkSession.builder.appName("FinalProjectYoutube").getOrCreate();
root_dir = "youtube-new/";
#read in datasets
ca_data = sqlContext.read.format("csv").option("header", "true").option("inferSchema", "true").load(root_dir + "CAvideos.csv");
de_data = sqlContext.read.format("csv").option("header", "true").option("inferSchema", "true").load(root_dir + "DEvideos.csv");
fr_data = sqlContext.read.format("csv").option("header", "true").option("inferSchema", "true").load(root_dir + "FRvideos.csv");
gb_data = sqlContext.read.format("csv").option("header", "true").option("inferSchema", "true").load(root_dir + "GBvideos.csv");
in_data = sqlContext.read.format("csv").option("header", "true").option("inferSchema", "true").load(root_dir + "INvideos.csv");
jp_data = sqlContext.read.format("csv").option("header", "true").option("inferSchema", "true").load(root_dir + "JPvideos.csv");
kr_data = sqlContext.read.format("csv").option("header", "true").option("inferSchema", "true").load(root_dir + "KRvideos.csv");
mx_data = sqlContext.read.format("csv").option("header", "true").option("inferSchema", "true").load(root_dir + "MXvideos.csv");
ru_data = sqlContext.read.format("csv").option("header", "true").option("inferSchema", "true").load(root_dir + "RUvideos.csv");
us_data = sqlContext.read.format("csv").option("header", "true").option("inferSchema", "true").load(root_dir + "USvideos.csv");

## Dataset Cleaning 

#### Enforce Datatypes

In [52]:
columns = ["video_id", "trending_date", "title", "channel_title", "category_id", "publish_time", 
           "tags", "views", "likes", "dislikes", "comment_count", "thumbnail_link", "comments_disabled", 
           "ratings_disabled", "video_error_or_removed", "description", "country"]


In [53]:
ca_data = castTypes_addCountry(ca_data, "Canada", columns)
de_data = castTypes_addCountry(de_data, "Germany", columns)
fr_data = castTypes_addCountry(fr_data, "France", columns)
gb_data = castTypes_addCountry(gb_data, "England", columns)
in_data = castTypes_addCountry(in_data, "India", columns)
jp_data = castTypes_addCountry(jp_data, "Japan", columns)
kr_data = castTypes_addCountry(kr_data, "South Korea", columns)
mx_data = castTypes_addCountry(mx_data, "Mexico", columns)
ru_data = castTypes_addCountry(ru_data, "Russia", columns)
us_data = castTypes_addCountry(us_data, "US", columns)

#### Handle Duplicates

In [54]:
# When dropping the duplicate rows based on video_id we found that half of the dataset gets dropped.  So we now look at the 
# duplicate rows to find out more information about what is going on.  Looking at the duplicate rows you can see that the 
# same video can be trending for mulitple days and in different countries causing the same video to exist in multiple rows.
# We decided the "duplicates" were not actually truely duplicate rows, the information provided by these multiple entries 
# is still useful.  We determined a truely duplicate row requires the same: "video_id", "views", "likes", "dislikes",
# "country", and "trending_date" column values.
two_data = us_data.union(in_data)

pandasdf = two_data.toPandas()

pandas_df = pandasdf.loc[pandasdf['video_id'].duplicated()]
pandas_df# = pandas_df[pandas_df['video_id'] != "\\n"]
#pandasdf[pandasdf['video_id'] == "d380meD0W0M"]

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country
124,\n,NaT,None,None,None,NaT,None,NaN,NaN,NaN,NaN,None,None,None,None,None,US
217,\n,NaT,None,None,None,NaT,None,NaN,NaN,NaN,NaN,None,None,None,None,None,US
226,\n,NaT,None,None,None,NaT,None,NaN,NaN,NaN,NaN,None,None,None,None,None,US
228,\n,NaT,None,None,None,NaT,None,NaN,NaN,NaN,NaN,None,None,None,None,None,US
242,kgaO45SyaO4,2017-11-15,The New SpotMini,BostonDynamics,28,2017-11-13,"""Robots""|""Boston Dynamics""|""SpotMini""|""Legged ...",3701763.0,56561.0,2495.0,11608.0,https://i.ytimg.com/vi/kgaO45SyaO4/default.jpg,False,False,False,For more information . . . stay tuned.,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86660,iNHecA3PJCo,2018-06-14,फेकू आशिक़ - राजस्थान की सबसे शानदार कॉमेडी | ...,RDC Rajasthani,23,2018-06-13,"""twinkle vaishnav comedy|""""twinkle vaishnav""""|...",214378.0,3291.0,404.0,196.0,https://i.ytimg.com/vi/iNHecA3PJCo/default.jpg,False,False,False,PRG Music & RDC Rajasthani presents फेकू आशिक़...,India
86661,dpPmPbhcslM,2018-06-14,Seetha | Flowers | Ep# 364,Flowers TV,24,2018-06-13,"""flowers serials|""""actress""""|""""malayalam seria...",406828.0,1726.0,478.0,1428.0,https://i.ytimg.com/vi/dpPmPbhcslM/default.jpg,False,False,False,"Flowers - A R Rahman Show,Book your Tickets He...",India
86662,mV6aztP58f8,2018-06-14,Bhramanam I Episode 87 - 12 June 2018 I Mazhav...,Mazhavil Manorama,24,2018-06-13,"""mazhavil manorama|""""bhramanam full episode""""|...",386319.0,1216.0,453.0,697.0,https://i.ytimg.com/vi/mV6aztP58f8/default.jpg,False,False,False,Subscribe to Mazhavil Manorama now for your da...,India
86663,qxqDNP1bDEw,2018-06-14,Nua Bohu | Full Ep 285 | 13th June 2018 | Odia...,Tarang TV,24,2018-06-13,"""tarang|""""tarang tv""""|""""tarang tv online""""|""""t...",130263.0,698.0,115.0,65.0,https://i.ytimg.com/vi/qxqDNP1bDEw/default.jpg,False,False,False,Nuabohu : Story of a rustic village girl who w...,India


In [55]:
compare_duplicate_cols = ["video_id", "views", "likes", "dislikes", "country", "trending_date"]
no_nan_cols = ["video_id", "trending_date", "title", 
        "channel_title", "category_id",  
        "tags", "views", "likes", "dislikes", "comment_count", 
        "thumbnail_link", "comments_disabled", "ratings_disabled", 
        "video_error_or_removed", "country"]
all_data = us_data.union(ca_data).union(de_data).union(fr_data).union(gb_data).union(in_data)
all_data = all_data.union(jp_data).union(kr_data).union(mx_data).union(ru_data).dropDuplicates(compare_duplicate_cols)
all_data.createOrReplaceTempView("all_data")
all_data = all_data.na.drop(subset=no_nan_cols) #drop nans from these columns
all_data.count() #with duplicates, count is 416869
#369492 with nans

363361

#### Handle NA's and Nulls

In [56]:
all_data.fillna("", subset="description")

DataFrame[video_id: string, trending_date: timestamp, title: string, channel_title: string, category_id: string, publish_time: timestamp, tags: string, views: int, likes: int, dislikes: int, comment_count: int, thumbnail_link: string, comments_disabled: boolean, ratings_disabled: boolean, video_error_or_removed: boolean, description: string, country: string]